In [1]:
import torch
from torchvision import datasets, transforms

model = torch.hub.load('pytorch/vision:v0.4.2', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.4.2


In [0]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [0]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

In [4]:
output.max(1)

torch.return_types.max(values=tensor([2.5147]), indices=tensor([463]))

In [5]:
torch.nn.functional.softmax(output[0], dim=0).max()

tensor(0.0087)

In [6]:
model.zero_grad()

output = model(input_batch)
y = torch.randn(output.size())
criterion = torch.nn.MSELoss()

loss = criterion(output, y)

print(loss)

loss.backward()

# print([param.grad for param in model.parameters()])

tensor(1.7182, grad_fn=<MseLossBackward>)


In [7]:
output.max(1)

torch.return_types.max(values=tensor([2.5147], grad_fn=<MaxBackward0>), indices=tensor([463]))

In [8]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])),
        batch_size=1, shuffle=True)

Files already downloaded and verified


In [9]:
train_loader.dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']